In [ ]:
# Function to save jump testing plots
def plot_jump_testing(df, athlete_id, pdf):
    """
    Generate and save jump testing plots for an athlete, grouped by Level.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): Athlete ID.
        pdf (PdfPages): PDF object to save the plot.

    Returns:
        None
    """
    # Retrieve athlete's info
    athlete_info = df[df['ID'] == athlete_id][['First Name', 'Last Name', 'Level']].iloc[0]
    athlete_name = f"{athlete_info['First Name']} {athlete_info['Last Name']}"
    athlete_level = athlete_info['Level']

    # Filter for Jump Tests and separate Broad and Vertical data
    jump_data = df[df['Test Type'] == 'Jump']
    broad_data = jump_data[jump_data['Test Sub-Type'] == 'Broad'][['ID', 'Test Number', 'Distance']]
    vertical_data = jump_data[jump_data['Test Sub-Type'] == 'Vertical'][['ID', 'Test Number', 'Vert']]

    # Ensure data copies before modifying
    broad_data = broad_data.copy()
    vertical_data = vertical_data.copy()

    # Convert to numeric and drop NaNs
    broad_data['Distance'] = pd.to_numeric(broad_data['Distance'], errors='coerce')
    vertical_data['Vert'] = pd.to_numeric(vertical_data['Vert'], errors='coerce')

    broad_data = broad_data.dropna(subset=['Distance'])
    vertical_data = vertical_data.dropna(subset=['Vert'])

    # Merge broad and vertical data on 'ID' and 'Test Number'
    merged_jump_data = pd.merge(broad_data, vertical_data, on=['ID', 'Test Number'], how='inner')

    # Get the athlete's jump test data
    athlete_jump_data = merged_jump_data[merged_jump_data['ID'] == athlete_id]
    if athlete_jump_data.empty:
        print("No jump test data found for the specified athlete.")
        return

    # Remove the athlete's data from facility-wide data to avoid overlap
    facility_data = merged_jump_data[merged_jump_data['ID'] != athlete_id]

    # Filter facility data by Level
    level_filtered_data = facility_data.merge(df[['ID', 'Level']], on='ID')
    level_filtered_data = level_filtered_data[level_filtered_data['Level'] == athlete_level]

    # Identify the athlete's most recent and other test data points
    highest_test_number = athlete_jump_data['Test Number'].max()
    specified_test = athlete_jump_data[athlete_jump_data['Test Number'] == highest_test_number]
    other_athlete_tests = athlete_jump_data[athlete_jump_data['Test Number'] != highest_test_number]

    # Create the plot
    fig, ax = plt.subplots(figsize=(14, 8))

    # Plot all facility-wide jump tests (excluding the specified athlete)
    ax.scatter(facility_data['Distance'], facility_data['Vert'], alpha=0.6, label='All Athletes', color='lightgray', marker='o')

    # Plot level-filtered data points (blue)
    ax.scatter(level_filtered_data['Distance'], level_filtered_data['Vert'], alpha=0.6, label=f"{athlete_level} Group", color='blue', marker='o')

    # Plot athlete's other tests (green)
    ax.scatter(other_athlete_tests['Distance'], other_athlete_tests['Vert'], alpha=0.8, color='green', label=f"Other Tests by {athlete_name}", marker='^')

    # Plot athlete's most recent test (red)
    ax.scatter(specified_test['Distance'], specified_test['Vert'], alpha=1.0, color='red', label=f"Most Recent Test by {athlete_name}", marker='D')

    # **Add Line of Best Fit for the Level Group (Blue Data)**
    if not level_filtered_data.empty:
        x = level_filtered_data['Distance']
        y = level_filtered_data['Vert']

        # Calculate linear regression coefficients (slope and intercept)
        slope, intercept = np.polyfit(x, y, 1)

        # Generate fitted y-values for the trend line
        x_range = np.linspace(x.min(), x.max(), 100)
        y_fit = slope * x_range + intercept

        # Plot the line of best fit
        ax.plot(x_range, y_fit, color='blue', linestyle='--', linewidth=2, label=f"{athlete_level} Group Trend")

    # Labels and title
    ax.set_xlabel("Broad Jump Distance (ft)")
    ax.set_ylabel("Vertical Jump Vert (in)")
    ax.set_title(f"Jump Tests for {athlete_level} and Athlete", fontsize=14, fontweight='bold')

    ax.legend()

    # Save the figure to the PDF
    pdf.savefig(fig)
    plt.close(fig)